# JustWatch와 IMDb 데이터 전처리

In [37]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import requests
import urllib.parse
import json

In [38]:
# 엑셀 파일을 읽어옵니다.
df = pd.read_excel('../data/imdb_final.xlsx', index_col=0)
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak..."
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm..."
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si..."
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani..."
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ..."


In [39]:
# 애니메이션, TV쇼 개별 삭제
df.drop(124, axis=0, inplace=True)
df.drop(151, axis=0, inplace=True)
df.drop(696, axis=0, inplace=True)
df.drop(826, axis=0, inplace=True)
df.drop(865, axis=0, inplace=True)
df.drop(873, axis=0, inplace=True)
df.drop(887, axis=0, inplace=True)
df.drop(913, axis=0, inplace=True)
df.drop(947, axis=0, inplace=True)
df.drop(1032, axis=0, inplace=True)
df.drop(1058, axis=0, inplace=True)

In [40]:
# '평점 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '평점 없음'].index

# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

# 시즌1 '평점 없음' 행 삭제 갯수
print(len(rows_to_drop))

153


In [41]:
# '2024년 공개작 포함'인 행들을 선택
rows_to_drop = df[df['season_1'] == '2024년 공개작 포함'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '2024년 공개작 포함' 행 삭제 갯수
len(rows_to_drop)

10

In [42]:
# '에피소드 정보 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '에피소드 정보 없음'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '에피소드 정보 없음' 행 삭제 갯수
print(len(rows_to_drop))

7


In [43]:
# 'season_1' 컬럼의 값이 None인 행들을 선택
rows_to_drop = df[df['season_1'].isna()].index

# 삭제된 행의 수 출력
print(len(rows_to_drop))

0


In [44]:
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

In [45]:
# production_country 결측치 채우고 영어 이름(china) 한국이름으로 바꾸기
df['Production country'] = df['Production country'].fillna('China')

In [46]:
# 번역할 나라 이름 딕셔너리
countries_dict = {'China': '중국', 'Arjantin': '아르헨티나', '필리핀 제도': '필리핀'}

# 각 행의 값을 번역하는 함수
def translate_country(country):
    countries = country.split(', ')
    translated_countries = (countries_dict.get(c, c) for c in countries)
    return ', '.join(translated_countries)

# 'country' 열에 번역 함수 적용
df['Production country'] = df['Production country'].apply(translate_country)

In [47]:
df['Production country'].unique()

array(['대한민국', '벨기에', '대만', '미국', '덴마크', '독일', '일본', '태국', '캐나다', '그리스',
       '영국', '스페인', '미국, 영국', '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국',
       '미국, 프랑스', '중국', '호주', '스웨덴', '이탈리아', '케냐', '인도', '호주, 미국', '스위스',
       '필리핀', '영국, 미국', '남아프리카공화국', '폴란드', '브라질', '미국, 캐나다', '콜롬비아', '터키',
       '룩셈부르크', '핀란드', '이스라엘', '프랑스', '멕시코', '벨기에, 네덜란드', '덴마크, 노르웨이',
       '노르웨이', '러시아', '미국, 일본', '영국, 프랑스', '요르단', '프랑스, 미국', '캐나다, 영국',
       '인도네시아', '영국, 인도', '아르헨티나', '아이슬란드', '오스트리아', '포르투갈',
       '프랑스, 독일, 스페인, 영국', '아일랜드, 영국', '캐나다, 프랑스, 독일', '이스라엘, 노르웨이',
       '대한민국, 미국', '미국, 대한민국', '네덜란드', '멕시코, 스페인', '칠레', '뉴질랜드, 호주',
       '사우디아라비아', '나이지리아', '이집트', '일본, 미국', '미국, 멕시코', '쿠웨이트',
       '나이지리아, 미국', '말레이시아, 대만', '사우디아라비아, 인도', '벨기에, 룩셈부르크',
       '사우디아라비아, 튀니지, 이집트, 모로코, 레바논', '독일, 오스트리아', '체코 공화국, 독일',
       '영국, 스페인', '스페인, 브라질', '레바논'], dtype=object)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 909 entries, 0 to 1089
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   original_title      726 non-null    object 
 2   year                909 non-null    int64  
 3   season_episode      909 non-null    int64  
 4   runtime             909 non-null    object 
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   Production country  909 non-null    object 
 8   IMDb_title          909 non-null    object 
 9   IMDb_URL            909 non-null    object 
 10  total_rate          909 non-null    float64
 11  total_count         909 non-null    object 
 12  watchlist           818 non-null    object 
 13  popularity          500 non-null    object 
 14  user_review         811 non-null    object 
 15  critic_review       781 non-null    float64
 16  age_miss    

In [49]:
# 엑셀 파일을 읽어옵니다.
df_us = pd.read_excel('../data/justwatch_us_age.xlsx', index_col=0)
df_us.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,add_original_title,add_english_title,justwatch_us_age
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021,https://www.imdb.com/title/tt10919420/,오리지날 타이틀 없음,Squid Game,TV-MA
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Mini Series 2022,https://www.imdb.com/title/tt13589004/,오리지날 타이틀 없음,The Twelve,TV-MA
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020,https://www.imdb.com/title/tt12079212/,誰是被害者,The Victims' Game,TV-MA
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021,https://www.imdb.com/title/tt12809988/,오리지날 타이틀 없음,Sweet Tooth,TV-14
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020,https://www.imdb.com/title/tt11612120/,오리지날 타이틀 없음,Sweet Home,TV-MA


In [50]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1089
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               1090 non-null   object
 1   original_title      855 non-null    object
 2   year                1090 non-null   int64 
 3   season_episode      1090 non-null   int64 
 4   runtime             1090 non-null   object
 5   genre               1090 non-null   object
 6   age_rating          803 non-null    object
 7   Production country  1089 non-null   object
 8   IMDb_title          1079 non-null   object
 9   IMDb_URL            1079 non-null   object
 10  add_original_title  1090 non-null   object
 11  add_english_title   1090 non-null   object
 12  justwatch_us_age    1090 non-null   object
dtypes: int64(2), object(11)
memory usage: 119.2+ KB


In [51]:
df_us.drop(['runtime', 'age_rating','Production country', 'IMDb_title','IMDb_URL'], axis=1, inplace=True)

In [52]:
df = pd.merge(df, df_us, on=['title','original_title','year','season_episode','genre'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   original_title      726 non-null    object 
 2   year                909 non-null    int64  
 3   season_episode      909 non-null    int64  
 4   runtime             909 non-null    object 
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   Production country  909 non-null    object 
 8   IMDb_title          909 non-null    object 
 9   IMDb_URL            909 non-null    object 
 10  total_rate          909 non-null    float64
 11  total_count         909 non-null    object 
 12  watchlist           818 non-null    object 
 13  popularity          500 non-null    object 
 14  user_review         811 non-null    object 
 15  critic_review       781 non-null    float64
 16  age_miss

In [53]:
# 컬럼명 rename
df.rename(columns={'Production country': 'production_country','season_episode':'total_season_num','IMDb_title':'imdb_title','IMDb_URL':'imdb_url', 'add_english_title':'english_title'}, inplace=True)

In [54]:
# 'min'을 '분'으로 바꾸는 함수
def replaceMin(string):
    return string.replace('min', '분')

# 시간을 분으로 변환하는 함수
def getTime(runtime):
    # '시간'과 '분'을 기준으로 분리
    hours, minutes = 0, 0
    if '시간' in runtime:
        hours = int(runtime.split('시간')[0].strip())
    if '분' in runtime:
        if '시간' in runtime:
            minutes = int(runtime.split('시간')[1].split('분')[0].strip())
        else:
            minutes = int(runtime.split('분')[0].strip())
    # 시간과 분을 합쳐서 분으로 계산
    total_minutes = hours * 60 + minutes
    return total_minutes

# 'min'을 '분'으로 변경
df['runtime'] = df['runtime'].apply(replaceMin)
# 'runtime' 컬럼의 값을 변환
df['runtime'] = df['runtime'].apply(getTime)

In [55]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

64
[


In [56]:
# 문자열을 리스트로 변환하는 함수
def parseStringToList(string):
    if pd.isna(string):  # NaN 값인 경우 빈 리스트 반환
        return None
    # 문자열에서 각 요소를 구분하는 쉼표로 분할
    elements = string.split(', ')
    # 각 요소에서 양쪽의 작은따옴표 제거
    elements = [element.strip("['").strip("']") for element in elements]
    return elements

# 'production','distributor' 컬럼의 값을 변환
df['production'] = df['production'].apply(parseStringToList)
df['distributor'] = df['distributor'].apply(parseStringToList)

In [57]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

2
Vlaamse Radio en Televisie (VRT)


In [58]:
# 'imdb_season_num' 컬럼 생성 및 초기화
df['imdb_season_num'] = np.nan

# 각 season 컬럼에 대해 순회하면서 해당 시즌이 없거나 '평점없음'인 경우에 해당하는 값으로 설정
for i in range(2, 18):
    season_col = f'season_{i}'
    condition = (df[season_col].isnull()) | (df[season_col] == '평점 없음')
    df.loc[condition & df['imdb_season_num'].isnull(), 'imdb_season_num'] = i - 1

# season_17에 데이터가 있는 경우 imdb_season_num을 17로 설정
if 'season_17' in df.columns and df['season_17'].notnull().any():
    df.loc[df['season_17'].notnull(), 'imdb_season_num'] = 17

# 결과 출력
print(df[['title', 'imdb_season_num']])

                       title  imdb_season_num
0               Ojingeo Geim              1.0
1                   12인의 심판자              2.0
2                     희생자 게임              1.0
3         스위트 투스: 사슴뿔을 가진 소년              2.0
4                 Sweet Home              2.0
..                       ...              ...
904                      레일라              1.0
905              100만 엔의 여인들              1.0
906                  킬러 코미디언              1.0
907                       달러              1.0
908  엘리트들, 못다 한 이야기: 나디아 구스만              1.0

[909 rows x 2 columns]


In [59]:
df.drop(['total_rate', 'total_count', 'watchlist','popularity', 'user_review', 'critic_review'], axis=1, inplace=True)

In [60]:
# 엑셀 파일을 읽어옵니다.
df_new = pd.read_excel('../data/IMDb_first_page_결측치_수정본_콤마제거.xlsx', index_col=0)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1079 entries, 0 to 1078
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1079 non-null   object 
 1   original_title      845 non-null    object 
 2   year                1079 non-null   int64  
 3   season_episode      1079 non-null   int64  
 4   runtime             1079 non-null   object 
 5   genre               1079 non-null   object 
 6   age_rating          798 non-null    object 
 7   Production country  1078 non-null   object 
 8   IMDb_title          1079 non-null   object 
 9   IMDb_URL            1079 non-null   object 
 10  total_rate          1077 non-null   float64
 11  total_count         1077 non-null   float64
 12  watchlist           1079 non-null   int64  
 13  popularity          1079 non-null   int64  
 14  user_review         1079 non-null   int64  
 15  critic_review       1079 non-null   int64  
 16  age_miss   

In [61]:
df_new.drop(['runtime', 'age_rating','Production country', 'IMDb_title','IMDb_URL','age_miss'], axis=1, inplace=True)

In [62]:
df = pd.merge(df, df_new, on=['title','original_title', 'year', 'genre'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   original_title      726 non-null    object 
 2   year                909 non-null    int64  
 3   total_season_num    909 non-null    int64  
 4   runtime             909 non-null    int64  
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   production_country  909 non-null    object 
 8   imdb_title          909 non-null    object 
 9   imdb_url            909 non-null    object 
 10  age_miss            865 non-null    object 
 11  production          826 non-null    object 
 12  distributor         902 non-null    object 
 13  season_1            909 non-null    object 
 14  season_2            295 non-null    object 
 15  season_3            145 non-null    object 
 16  season_4

In [63]:
df.head()

,title,original_title,year,total_season_num,runtime,genre,age_rating,production_country,imdb_title,imdb_url,...,english_title,justwatch_us_age,imdb_season_num,season_episode,total_rate,total_count,watchlist,popularity,user_review,critic_review
0,Ojingeo Geim,Squid Game,2021,2,55,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,Squid Game,TV-MA,1.0,2,8.0,549000.0,250000,237,2600,109
1,12인의 심판자,The Twelve,2019,2,54,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,The Twelve,TV-MA,2.0,2,7.4,6500.0,9800,2530,89,7
2,희생자 게임,誰是被害者,2020,2,60,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,The Victims' Game,TV-MA,1.0,2,7.3,1900.0,2100,0,25,0
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,Sweet Tooth,TV-14,2.0,3,7.7,68000.0,97000,41,791,86
4,Sweet Home,NaN,2020,3,60,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,Sweet Home,TV-MA,2.0,3,7.3,33000.0,43000,940,516,37


In [64]:
# 새로운 컬럼 순서 지정
columns = [
    'title', 'english_title', 'year', 'imdb_season_num', 'runtime', 'genre',
    'age_rating', 'age_miss','justwatch_us_age', 'production_country',
    'total_count', 'total_rate', 'watchlist', 'popularity', 'user_review', 'critic_review',
    'production', 'distributor', 'season_1', 'season_2', 'season_3', 'season_4',
    'season_5', 'season_6', 'season_7', 'season_8', 'season_9', 'season_10',
    'season_11', 'season_12', 'season_13', 'season_14', 'season_15', 'season_16',
    'season_17', 'director', 'writer', 'actor',
    'imdb_title', 'imdb_url'
]
# 데이터프레임의 컬럼 순서 변경
df = df[columns]

In [65]:
# 컬럼명 rename
df.rename(columns={'imdb_season_num':'total_season_num'}, inplace=True)

### '1899'만 merge가 안되어 개별적으로 채움

In [66]:
# english_title 결측치 채우기
df['english_title'] = df['english_title'].fillna('1899')

In [67]:
# justwatch_us_age 결측치 채우기
df['justwatch_us_age'] = df['justwatch_us_age'].fillna('TV-MA')

### season데이터 고치기

In [68]:
import ast

# 문자열을 딕셔너리로 변환하는 함수
def preprocess_data(data_str):
    try:
        data_dict = ast.literal_eval(data_str)
        if not data_dict:  # 빈 딕셔너리인 경우
            return None
        else:
            return data_dict
    except (ValueError, SyntaxError):
        return None  # 오류 발생 시 None 반환

# K와 M 값을 실제 숫자로 변환하는 함수
def convert_views(episode_data):
    if episode_data is None:  # None인 경우 처리
        return None
    
    converted_data = {}
    for episode, values in episode_data.items():
        rating = float(values[0])  # rating 값을 실수로 변환
        views = values[1]
        if views.isdigit():
            views = int(views)
        elif views.endswith('K'):
            views = int(float(views[:-1]) * 1000)
        elif views.endswith('M'):
            views = int(float(views[:-1]) * 1000000)
        converted_data[episode] = [rating, views]
    return converted_data

# season_으로 시작하는 컬럼에 대해서만 변환 적용
for column in df.columns:
    if column.startswith('season_'):
        df[column] = df[column].apply(preprocess_data).apply(convert_views)

# 변환된 데이터프레임 출력
print(df)


                       title                      english_title  year  \
0               Ojingeo Geim                         Squid Game  2021   
1                   12인의 심판자                         The Twelve  2019   
2                     희생자 게임                  The Victims' Game  2020   
3         스위트 투스: 사슴뿔을 가진 소년                        Sweet Tooth  2021   
4                 Sweet Home                         Sweet Home  2020   
..                       ...                                ...   ...   
904                      레일라                              Leila  2019   
905              100만 엔의 여인들                  Million Yen Women  2017   
906                  킬러 코미디언                            Hasmukh  2020   
907                       달러                             Dollar  2019   
908  엘리트들, 못다 한 이야기: 나디아 구스만  Elite Short Stories: Nadia Guzmán  2021   

     total_season_num  runtime                      genre age_rating age_miss  \
0                 1.0       55            

In [69]:
# import ast

# # 문자열을 딕셔너리로 변환하는 함수
# def preprocess_data(data_str):
#     try:
#         return ast.literal_eval(data_str)
#     except (ValueError, SyntaxError):
#         return {}  # 오류 발생 시 빈 딕셔너리 반환

# # K와 M 값을 실제 숫자로 변환하는 함수
# def convert_views(episode_data):
#     converted_data = {}
#     for episode, values in episode_data.items():
#         rating = float(values[0])  # rating 값을 실수로 변환
#         views = values[1]
#         if views.isdigit():
#             views = int(views)
#         elif views.endswith('K'):
#             views = int(float(views[:-1]) * 1000)
#         elif views.endswith('M'):
#             views = int(float(views[:-1]) * 1000000)
#         converted_data[episode] = [rating, views]
#     return converted_data

# # season_으로 시작하는 컬럼에 대해서만 변환 적용
# for column in df.columns:
#     if column.startswith('season_'):
#         df[column] = df[column].apply(preprocess_data).apply(convert_views)

In [70]:
df['season_1']

0      {'S1.E1': [8.2, 16000], 'S1.E2': [7.5, 14000],...
1      {'S1.E1': [7.3, 224], 'S1.E2': [7.2, 199], 'S1...
2      {'S1.E1': [7.4, 62], 'S1.E2': [7.6, 56], 'S1.E...
3      {'S1.E1': [7.9, 2500], 'S1.E2': [8.1, 2100], '...
4      {'S1.E1': [7.1, 1100], 'S1.E2': [7.2, 929], 'S...
                             ...                        
904    {'S1.E1': [5.6, 322], 'S1.E2': [5.8, 277], 'S1...
905    {'S1.E1': [7.6, 148], 'S1.E2': [7.2, 43], 'S1....
906    {'S1.E1': [5.8, 60], 'S1.E2': [5.5, 45], 'S1.E...
907    {'S1.E1': [8.0, 44], 'S1.E2': [8.0, 33], 'S1.E...
908    {'S1.E1': [6.5, 225], 'S1.E2': [6.5, 221], 'S1...
Name: season_1, Length: 909, dtype: object

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               909 non-null    object 
 1   english_title       909 non-null    object 
 2   year                909 non-null    int64  
 3   total_season_num    909 non-null    float64
 4   runtime             909 non-null    int64  
 5   genre               909 non-null    object 
 6   age_rating          653 non-null    object 
 7   age_miss            865 non-null    object 
 8   justwatch_us_age    909 non-null    object 
 9   production_country  909 non-null    object 
 10  total_count         909 non-null    float64
 11  total_rate          909 non-null    float64
 12  watchlist           909 non-null    int64  
 13  popularity          909 non-null    int64  
 14  user_review         909 non-null    int64  
 15  critic_review       909 non-null    int64  
 16  producti

In [72]:
# # 문자열을 딕셔너리로 변환하는 함수
# def preprocess_data(data_str):
#     try:
#         data_dict = ast.literal_eval(data_str)
#         if not data_dict:  # 빈 딕셔너리인 경우
#             return None
#         else:
#             return data_dict
#     except (ValueError, SyntaxError):
#         return None  # 오류 발생 시 None 반환
    
# # season_으로 시작하는 컬럼에 대해서만 변환 적용
# for column in df.columns:
#     if column.startswith('season_'):
#         df[column] = df[column].apply(preprocess_data).apply(convert_views)

In [74]:
# 저장
df.to_csv('../data/imdb_final_3.csv', sep="/", encoding="utf-8")

df.to_excel('../data/imdb_final_3.xlsx')